In [ ]:
! pip install langchain langchain-community langchain-openai promptquality openai pinecone-client --upgrade

In [21]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [22]:
import pandas as pd

questions = pd.read_csv("eval_questions.csv")["questions"].tolist()

In [ ]:
import promptquality as pq

pq.login("ENTER_YOUR_GALILE_ENVIRONMENT_HERE")

In [23]:
# VARIABLES WE WILL BE EXPERIMENTING WITH TODAY
CHUNK_SIZE = 400
EMBEDDING_MODEL = "text-embedding-3-small"
TOP_K = 4
CHAT_MODEL = "gpt-3.5-turbo-0125"

# Index Name
index_name = f"beauty-rc-cs{CHUNK_SIZE}-{EMBEDDING_MODEL}"

In [24]:
def create_tags():
    return [
        pq.RunTag(key="VectorDB Index", value=index_name, tag_type=pq.TagType.RAG),
        pq.RunTag(key="Encoder", value=EMBEDDING_MODEL, tag_type=pq.TagType.RAG),
        pq.RunTag(key="LLM", value=CHAT_MODEL, tag_type=pq.TagType.RAG),
        pq.RunTag(key="Top k", value=str(TOP_K), tag_type=pq.TagType.RAG)
    ]

In [25]:
from promptquality import Scorers

metrics = [
    # RAG Quality Metrics
    Scorers.context_adherence,
    Scorers.completeness_gpt,
    Scorers.chunk_attribution_utilization_gpt,
    # Other metrics I care about
    Scorers.latency,
    Scorers.pii,
    Scorers.toxicity,
    Scorers.tone,
]

In [27]:
from qa_chain import get_qa_chain
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
my_app = get_qa_chain(embeddings, index_name, TOP_K, CHAT_MODEL, 0.1)


In [28]:
# Experiment Project and Run Names
project_name = "feb13-qa-pinecone"
run_name = f"{index_name}-{CHAT_MODEL}-k{TOP_K}"

In [29]:
galileo_callback = pq.GalileoPromptCallback(project_name=project_name, 
                                            run_name=run_name, 
                                            scorers=metrics, 
                                            run_tags=create_tags())

for question in questions:
    my_app.invoke(question, config=dict(callbacks=[galileo_callback]))
galileo_callback.finish()

Processing complete!: 100%|██████████| 5/5 [00:04<00:00,  1.20it/s]   

🔭 View your prompt run on the Galileo console at: https://console.demo.rungalileo.io/prompt/chains/f01ff29c-51d0-4ce7-b539-e1b83fda8148/31f874a8-3eba-4a94-b7dc-c45d517253fc?taskType=12
